In [1]:
import numpy as np
import jax.numpy as jnp
import jax
import cyipopt

from functools import partial
from typing import *
from dataclasses import dataclass, field
from jaxlie import SE3, SO3
import jax_dataclasses as jdc

from sdf_world.sdf_world import *
from sdf_world.robots import *
from sdf_world.util import *
from sdf_world.network import *
from sdf_world.sparse_ipopt import *

import time

In [2]:
world = SDFWorld()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [3]:
class PandaHand:
    def __init__(self, hand_model, name="hand", color="white"):
        self.model = hand_model
        self.robot = Robot(world.vis, name, hand_model, color=color, alpha=0.5, visualized_mesh="visual")
        self.hand_pose_wrt_ee = SE3.from_translation(jnp.array([0,0,-0.105]))
        self.hand_pc = self.robot.get_surface_points_fn(jnp.array([0.04, 0.04]))
    
    def get_bounding_box(self, name):
        fks = self.model.fk_fn(jnp.array([0.04, 0.04]))
        hand_points = []
        for i, link in enumerate(self.model.links.values()):
            pts = jax.vmap(SE3(fks[i]).apply)(link.collision_mesh.vertices)
            hand_points.append(pts)
        hand_points = np.vstack(hand_points, dtype=float)
        min_points = hand_points.min(axis=0)
        max_points = hand_points.max(axis=0)
        extents = max_points - min_points
        center = (max_points + min_points) / 2
        box = Box(world.vis, name, extents, alpha=0.5, visualize=False)
        box.set_translate(center)
        return box
    
    def set_pose(self, pose):
        self.robot.set_pose(pose @ self.hand_pose_wrt_ee)

In [4]:
# Load robot
hand_model = RobotModel(HAND_URDF, PANDA_PACKAGE, True)
for link_name, link in hand_model.links.items():
    link.set_surface_points(10)
hand1 = PandaHand(hand_model, "hand1", "blue")
hand2 = PandaHand(hand_model, "hand2", "blue")

In [5]:
hand3 = PandaHand(hand_model, "hand3", "green")
hand4 = PandaHand(hand_model, "hand4", "green")

In [6]:
#load sdf meshes
table_lengths = [0.4, 0.4, 0.2]
table_start = Box(world.vis, "table_start", table_lengths, 'white', 0.5)
table_goal = Box(world.vis, "table_goal", table_lengths, 'white', 0.5)
obstacle = Box(world.vis, "obstacle", [0.4, 0.2, 0.35], 'white', 0.5)

obj_start = Mesh(world.vis, "obj_start", 
                 "./sdf_world/assets/object/mesh.obj",
                 color="blue", alpha=0.5)
obj_goal = Mesh(world.vis, "obj_goal", 
                "./sdf_world/assets/object/mesh.obj",
                color="green", alpha=0.5)
obj_ho = Mesh(world.vis, "obj_ho", 
                "./sdf_world/assets/object/mesh.obj",
                color="yellow", alpha=0.5)
obj = Mesh(world.vis, "obj", 
                "./sdf_world/assets/object/mesh.obj",
                color="white", alpha=0.8)

In [7]:
ydev = 0.4


table_start.set_translate([0.45, -ydev, 0.2/2])
table_goal.set_translate([0.45, ydev, 0.2/2])
obj_lengths = obj_start.mesh.bounding_box.primitive.extents
obj_start.set_translate([0.45, -0.4, obj_lengths[-1]/2+table_lengths[-1]])
trans_goal = jnp.array([0.45, 0.4, obj_lengths[-2]/2+table_lengths[-1]])
obstacle.set_translate([0.45, 0., 0.35/2])
obj_goal_pose = SE3.from_rotation_and_translation(
    SO3.from_rpy_radians(jnp.pi/2, 0,0), trans_goal)
obj_goal.set_pose(obj_goal_pose)
obj.set_translate([0.45, -0.4, obj_lengths[-1]/2+table_lengths[-1]])
obj_ho.set_translate([0.3, 0., 0.6])

In [8]:
# constants, data
safe_dist = 0.05
ws_lb = np.array([-1,-1,-0.5, -np.pi, -np.pi, -np.pi])
ws_ub = np.array([1,1,1.5, np.pi, np.pi, np.pi])
hand_pose_wrt_ee = SE3.from_translation(jnp.array([0,0,-0.105]))
hand_pc = hand1.hand_pc

#links_points_mat
# links_points_mat = []
# for link in panda_model.links.values():
#     if not link.has_mesh: continue
#     if not panda_model.is_floating and link == panda_model.root_link: continue
#     links_points_mat.append(link.surface_points)
# links_points_mat = np.array(links_points_mat)

#object points
#obj_points = farthest_point_sampling(obj_start.mesh.sample(200), 20)

# environment sdf
env = SDFContainer([table_start, table_goal, obstacle], safe_dist)

In [9]:
# models 
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
restored_grasp = orbax_checkpointer.restore("model/grasp_net_prob_dist")
restored_manip = orbax_checkpointer.restore("model/manip_net")

#grasp net
grasp_net = GraspNet(32)
grasp_fn = lambda x: grasp_net.apply(restored_grasp["params"], x)

grasp_logit_fn = lambda g: grasp_fn(g)[0]
grasp_dist_fn = lambda g: grasp_fn(g)[1]

#manip net: input: wxyzxyz
manip_net = ManipNet(64)
zflip = SE3.from_rotation(SO3.from_z_radians(jnp.pi))
def manip_fn(wxyzxyz:Array):
    x1 = wxyzxyz.at[:4].set(SO3(wxyzxyz[:4]).normalize().parameters())
    x2 = (SE3(x1) @ zflip).parameters()
    return jax.vmap(manip_net.apply, (None, 0))(
        restored_manip["params"], jnp.vstack([x1, x2])).flatten()

In [10]:
#functions
def grasp_reconst(grasp:Array):
    rot = SO3(grasp_fn(grasp)[2:]).normalize()
    trans = grasp/restored_grasp["scale_to_norm"]
    return SE3.from_rotation_and_translation(rot, trans)
def grasp_embedding(grasp_point):
    grasp = grasp_point * restored_grasp["scale_to_norm"]
    return grasp

In [11]:
#utility functions
to_posevec = lambda wxyzxyz: jnp.hstack([wxyzxyz[4:], SO3(wxyzxyz[:4]).log()])
to_wxyzxyz = lambda posevec: jnp.hstack([SO3.exp(posevec[3:]).parameters(), posevec[:3]])
to_pose = lambda posevec: SE3(to_wxyzxyz(posevec))
def split_pos_qtn(pose):
    return pose.parameters()[-3:], pose.parameters()[:4]
def merge_pose(pos, qtn):
    return SE3(jnp.hstack([qtn, pos])).normalize()

In [12]:
def grasp_constr(g):
    return grasp_logit_fn(g)
jac_grasp_constr = jax.grad(grasp_constr, argnums=[0])

In [13]:
# def manip_constr_ec(grasp, posevec_ho, pose_o, pose_r, pose_oref):
#     pose_ho = SE3(pose_oref) @ to_pose(posevec_ho)
#     grasp_pose1 = SE3(pose_r).inverse() @ pose_ho @ grasp_reconst(grasp)
#     grasp_pose2 = SE3(pose_r).inverse() @ SE3(pose_o) @ grasp_reconst(grasp)
#     posevecs = [pose.parameters() for pose in [grasp_pose1, grasp_pose2]]
#     manips = jax.vmap(manip_fn)(jnp.vstack(posevecs))
#     is_flip = manips.min(axis=0).argmax()
#     return manips[:,is_flip]
# jac_manip_constr = jax.jacfwd(manip_constr_ec, argnums=[0, 1])
# jac_manip_constr = jax.jit(jac_manip_constr)

In [14]:
def manip_constr_qtn(grasp, pose_ho, pose_o, pose_r):
    grasp_pose1 = SE3(pose_r).inverse() @ SE3(pose_ho).normalize() @ grasp_reconst(grasp)
    grasp_pose2 = SE3(pose_r).inverse() @ SE3(pose_o) @ grasp_reconst(grasp)
    posevecs = [pose.parameters() for pose in [grasp_pose1, grasp_pose2]]
    manips = jax.vmap(manip_fn)(jnp.vstack(posevecs))
    is_flip = manips.min(axis=0).argmax()
    return manips[:,is_flip]
jac_manip_constr_qtn = jax.jacfwd(manip_constr_qtn, argnums=[0, 1])
jac_manip_constr_qtn = jax.jit(jac_manip_constr_qtn)

In [15]:
def qtn_constr(pose_like):
    qtn_like = pose_like[:4]
    return jnp.sum(qtn_like**2) - 1
jac_qtn_constr = jax.grad(qtn_constr, argnums=[0])

In [16]:
def get_jac_indices(cdim, xdim, row_offset=0):
    rowcol = np.indices((cdim, xdim)).reshape(2,-1)
    rowcol[0] += row_offset
    return rowcol

def get_hand_pc(grasp, obj_pose):
    grasp_pose = grasp_reconst(grasp)
    hand_base_pose_wrt_world = SE3(obj_pose) @ grasp_pose @ hand_pose_wrt_ee
    assigned_hand_pc = jax.vmap(hand_base_pose_wrt_world.apply)(hand_pc)
    return assigned_hand_pc

def hand_col_constr(grasp, obj_pose):
    obj_pose = SE3(obj_pose).normalize().parameters()
    pc = get_hand_pc(grasp, obj_pose)
    distances = env.distances(pc)
    return distances.min()
jac_hand_col_constr = jax.jacfwd(hand_col_constr, argnums=[0,1])

In [17]:
hand_bb = hand1.get_bounding_box("hand_bb")
def hands_col_constr(grasp1, grasp2):
    grasp_pose1 = grasp_reconst(grasp1) @ hand1.hand_pose_wrt_ee
    grasp_pose2 = grasp_reconst(grasp2) @ hand1.hand_pose_wrt_ee
    grasp1_wrt_2 = grasp_pose1.inverse() @ grasp_pose2
    points = jax.vmap(grasp1_wrt_2.apply)(hand_pc)
    return jax.vmap(hand_bb.sdf)(points).min()
jac_hands_col_constr = jax.jacfwd(hands_col_constr, argnums=[0,1])

In [18]:
def centering(pose_ho):
    pos = pose_ho[-3:]
    pos_err = obj_ho.pose.translation() - pos
    return jnp.sum(pos_err **2)
grad_centering = jax.grad(centering, argnums=[0])

In [19]:
ho_qtn_lb = np.array([-1,-1,-1,-1, 0., -0.4, 0.])
ho_qtn_ub = np.array([1, 1,1,1, 0.5, 0.4, 0.7])

In [20]:
bdr1 = SparseIPOPT()
bdr1.add_variable("grasp_0", 3, -1., 1.)
bdr1.add_variable("grasp_1", 3, -1., 1.)
bdr1.add_variable("pose_ho", 7, ho_qtn_lb, ho_qtn_ub)
#bdr1.add_parameter("pose_ho", 7)
bdr1.add_parameter("pose_o0", 7)
bdr1.add_parameter("pose_og", 7)
bdr1.add_parameter("pose_r1", 7)
bdr1.add_parameter("pose_r2", 7)

bdr1.register_fn("grasp_fn", [3], 1,
    grasp_constr, jac_grasp_constr)
bdr1.register_fn("manip_fn", [3, 7, 7, 7], 2,
    manip_constr_qtn, jac_manip_constr_qtn, jac_out_argnums=[0, 1])
bdr1.register_fn("qtn_constr", [7], 1,
    qtn_constr, jac_qtn_constr)
# bdr1.register_fn("hand_col_constr", [3,7,7], 2, 
#                  hand_col_constr, jac_hand_col_constr,
#                  jac_out_argnums=[0,1],
#                  custom_jac_indices=jac_hand_col_constr_idx)
bdr1.register_fn("hand_col_constr", [3,7], 1, 
                 hand_col_constr, jac_hand_col_constr)
bdr1.register_fn("hands_col_constr", [3,3], 1, 
                 hands_col_constr, jac_hands_col_constr)
# bdr1.register_fn("obj", [7], 1, 
#                  centering, grad_centering)

In [21]:
#bdr1.set_objective("obj", ["pose_ho"])
bdr1.set_constr("grasp_prob_pick", "grasp_fn", ["grasp_0"], 
                   1., np.inf)
bdr1.set_constr("grasp_prob_place", "grasp_fn", ["grasp_1"], 
                   1., np.inf)
bdr1.set_constr("manip_pick_ho", "manip_fn", 
                ["grasp_0", "pose_ho", "pose_o0", "pose_r1"], 
                 0.2, np.inf) #, no_deriv_names=["grasp_0"]
bdr1.set_constr("manip_place_ho", "manip_fn", 
                ["grasp_1", "pose_ho", "pose_og", "pose_r2"], 
                 0.2, np.inf)

bdr1.set_constr("hand_col_pick", "hand_col_constr",
                ["grasp_0", "pose_o0"],
                0.05, np.inf)
bdr1.set_constr("hand_col_place", "hand_col_constr",
                ["grasp_1", "pose_og"],
                0.05, np.inf)
bdr1.set_constr("hand_col_ho_pick", "hand_col_constr",
                ["grasp_0", "pose_ho"],
                0.05, np.inf)
bdr1.set_constr("hand_col_ho_place", "hand_col_constr",
                ["grasp_1", "pose_ho"],
                0.05, np.inf)
bdr1.set_constr("hands_col", "hands_col_constr",
                ["grasp_0", "grasp_1"],
                0.05, np.inf)
# bdr1.set_constr("hand_col_1", "hand_col_constr",
#                 ["grasp_1", "pose_ho", "pose_og"],
#                 0.05, np.inf)
bdr1.set_constr("qtn_manifold", "qtn_constr", ["pose_ho"], 0., 0.)

In [22]:
pc_hand = PointCloud(world.vis, "hand_pc", np.zeros((10,3)), 0.01, "red")

In [23]:
def debug_callback(xdict):
    grasp0 = xdict["grasp_0"]
    grasp1 = xdict["grasp_1"]

    pose_ho = SE3(xdict["pose_ho"]).normalize()
    pose_o0 = SE3(xdict["pose_o0"])
    pose_og = SE3(xdict["pose_og"])
    obj_poses = jnp.vstack([
        pose_ho.parameters(), 
        pose_ho.parameters(),
        pose_o0.parameters(),
        pose_og.parameters()])
    grasps = jnp.vstack([
        grasp0,
        grasp1,
        grasp0,
        grasp1
    ])
    pcs = jax.vmap(get_hand_pc, in_axes=(0,0))(grasps, obj_poses).reshape(-1,3)
    pc_hand.reload(points=pcs)
    #print(f"pos:{pose_ho.parameters()[-3:]}")    
    obj.set_pose(pose_ho)
    grasp_pose1_ho0 = pose_ho @ grasp_reconst(grasp0)
    grasp_pose1_o0 = pose_o0 @ grasp_reconst(grasp0)
    grasp_pose1_ho1 = pose_ho @ grasp_reconst(grasp1)
    grasp_pose1_o1 = pose_og @ grasp_reconst(grasp1)
    hand1.set_pose(grasp_pose1_ho0)
    hand2.set_pose(grasp_pose1_o0)
    hand3.set_pose(grasp_pose1_ho1)
    hand4.set_pose(grasp_pose1_o1)
    #print(f"err: {qtn_err(qtn, qtn_d)}  viol:{qtn_constr(qtn)}")
    time.sleep(0.5)
bdr1.set_debug_callback(debug_callback)

In [24]:
ipopt_grasp = bdr1.build()

compiling objective ...
compiling gradient ...
compiling constraints ...
compiling jacobian ...
ooo--------------------------------------
---ooo-----------------------------------
ooo---ooooooo----------------------------
ooo---ooooooo----------------------------
---oooooooooo----------------------------
---oooooooooo----------------------------
ooo--------------------------------------
---ooo-----------------------------------
ooo---ooooooo----------------------------
---oooooooooo----------------------------
oooooo-----------------------------------
------ooooooo----------------------------


In [31]:
bdr1.print_x_info()

{'grasp_0': 3, 'grasp_1': 3, 'pose_ho': 7, 'pose_o0': 7, 'pose_og': 7, 'pose_r1': 7, 'pose_r2': 7}


In [25]:
grasp1 = np.random.normal(size=3) * 0.5
grasp2 = np.random.normal(size=3) * 0.5
pose_o = obj_start.pose.parameters()
pose_g = obj_goal.pose.parameters()

pose_r1 = SE3.from_translation(jnp.array([0, -0.4, 0])).parameters()
pose_r2 = SE3.from_translation(jnp.array([0, 0.4, 0])).parameters()
pose_oref = pose_o
pose_ho = obj_ho.pose.parameters()
x0 = np.hstack([grasp1, grasp2, pose_ho, pose_o, pose_g, pose_r1, pose_r2])
grasp_sol, info = ipopt_grasp.solve(x0)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        7
Number of nonzeros in inequality constraint Jacobian.:       78
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       41
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       13
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality c

### Robot

In [26]:
# robot, hand
panda_model = RobotModel(PANDA_URDF, PANDA_PACKAGE)
pandas = []
pandas.append(
    Robot(world.vis, "panda1", panda_model, color="blue", alpha=0.5)
)
pandas.append(
    Robot(world.vis, "panda2", panda_model, color="blue", alpha=0.5)
)
pandas.append(
    Robot(world.vis, "panda3", panda_model, color="green", alpha=0.5)
)
pandas.append(
    Robot(world.vis, "panda4", panda_model, color="green", alpha=0.5)
)

for panda in pandas:
    panda.reduce_dim([7, 8], [0.04, 0.04])

In [31]:
pandas[0].set_translate([0,-ydev,0])
pandas[1].set_translate([0,-ydev,0])
pandas[2].set_translate([0,ydev,0])
pandas[3].set_translate([0,ydev,0])

In [27]:
# kinematics
def skew(v):
    v1, v2, v3 = v
    return jnp.array([[0, -v3, v2],
                      [v3, 0., -v1],
                      [-v2, v1, 0.]])

@jax.custom_jvp
def fk(q):
    fks = panda_model.fk_fn(q)
    return fks[-1]

@fk.defjvp
def fk_jvp(primals, tangents):
    q, = primals
    q_dot, = tangents
    fks = panda_model.fk_fn(q)
    qtn, p_ee = fks[-1][:4], fks[-1][-3:]
    w, xyz = qtn[0], qtn[1:]
    geom_jac = []
    for posevec in fks[1:8]:
        p_frame = posevec[-3:]
        rot_axis = SE3(posevec).as_matrix()[:3, 2]
        lin_vel = jnp.cross(rot_axis, p_ee - p_frame)
        geom_jac.append(jnp.hstack([lin_vel, rot_axis]))
    geom_jac = jnp.array(geom_jac).T  #geom_jacobian
    H = jnp.hstack([-xyz[:,None], skew(xyz)+jnp.eye(3)*w])
    rot_jac = 0.5*H.T@geom_jac[3:,:]
    jac = jnp.vstack([rot_jac, geom_jac[:3,:]])
    return fks[-1], jac@q_dot

In [28]:
def kin_constr(q, grasp, obj_pose):
    grasp_pose = SE3(obj_pose) @ grasp_reconst(grasp)
    ee_pose = SE3(fk(q))
    pos_err = grasp_pose.translation() - ee_pose.translation()
    rot_err = (ee_pose.rotation().inverse() @ grasp_pose.rotation()).log()
    return jnp.hstack([pos_err, 0.5*rot_err])
jac_kin_constr = jax.jacfwd(kin_constr, argnums=[0])

In [29]:
q_lb, q_ub = pandas[0].lb, pandas[0].ub

bdr2 = SparseIPOPT()
bdr2.add_variable("q_pick", 7, q_lb, q_ub)
bdr2.add_variable("q_ho_1", 7, q_lb, q_ub)
bdr2.add_variable("q_ho_2", 7, q_lb, q_ub)
bdr2.add_variable("q_place", 7, q_lb, q_ub)
bdr2.add_parameter("grasp_0", 3)
bdr2.add_parameter("grasp_1", 3)
bdr2.add_parameter("pose_o0", 7)
bdr2.add_parameter("pose_ho", 7)
bdr2.add_parameter("pose_og", 7)
bdr2.add_parameter("pose_r1", 7)
bdr2.add_parameter("pose_r2", 7)

bdr2.register_fn("kin_fn", [7, 3, 7], 6,
    kin_constr, jac_kin_constr, jac_out_argnums=[0])

bdr2.set_constr("kin_pick", "kin_fn", 
                ["q_pick", "grasp_0", "pose_o0"],
                0., 0.)
bdr2.set_constr("kin_ho1", "kin_fn", ["q_ho_1", "grasp_0", "pose_ho"],
                0., 0.)
bdr2.set_constr("kin_ho2", "kin_fn", ["q_ho_2", "grasp_1", "pose_ho"],
                0., 0.)
bdr2.set_constr("kin_place", "kin_fn", ["q_place", "grasp_1", "pose_og"],
                0., 0.)


In [30]:
ipopt_ik = bdr2.build()

compiling objective ...
compiling gradient ...
compiling constraints ...
compiling jacobian ...


2023-08-11 16:37:30.134520: W external/xla/xla/service/gpu/ir_emitter_triton.cc:761] Shared memory size limit exceeded.


ooooooo------------------------------------------------
ooooooo------------------------------------------------
ooooooo------------------------------------------------
ooooooo------------------------------------------------
ooooooo------------------------------------------------
ooooooo------------------------------------------------
-------ooooooo-----------------------------------------
-------ooooooo-----------------------------------------
-------ooooooo-----------------------------------------
-------ooooooo-----------------------------------------
-------ooooooo-----------------------------------------
-------ooooooo-----------------------------------------
--------------ooooooo----------------------------------
--------------ooooooo----------------------------------
--------------ooooooo----------------------------------
--------------ooooooo----------------------------------
--------------ooooooo----------------------------------
--------------ooooooo---------------------------

In [63]:
bdr2.print_x_info()

{'q_pick': 7, 'q_ho_1': 7, 'q_ho_2': 7, 'q_place': 7, 'grasp_0': 3, 'grasp_1': 3, 'pose_o0': 7, 'pose_ho': 7, 'pose_og': 7}


In [65]:
q_init = pandas[0].neutral
grasp1 = bdr1.split_solution(grasp_sol)["grasp_0"]
grasp2 = bdr1.split_solution(grasp_sol)["grasp_1"]
pose_o0 = bdr1.split_solution(grasp_sol)["pose_o0"]
pose_ho = bdr1.split_solution(grasp_sol)["pose_ho"]
pose_og = bdr1.split_solution(grasp_sol)["pose_og"]
x0 = np.hstack([q_init,q_init,q_init,q_init, grasp1, grasp2, pose_o0, pose_ho, pose_og])
ik_sol, info = ipopt_ik.solve(x0)

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:      168
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       55
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       28
                     variables with only upper bounds:        0
Total number of equality constraints.................:       24
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 9.58e-01 0.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [68]:
pandas[0].set_joint_angles(bdr2.split_solution(ik_sol)["q_pick"])
pandas[1].set_joint_angles(bdr2.split_solution(ik_sol)["q_ho_1"])

In [72]:
pandas[2].set_joint_angles(bdr2.split_solution(ik_sol)["q_ho_2"])
pandas[3].set_joint_angles(bdr2.split_solution(ik_sol)["q_place"])

In [ ]:
ho_lb = np.array([0., -0.4, 0., -np.pi, -np.pi, -np.pi])
ho_ub = np.array([0.5, 0.4, 0.7, np.pi, np.pi, np.pi])

In [137]:
bdr1 = SparseIPOPT()
bdr1.add_variable("grasp_0", 3, -1., 1.)
bdr1.add_variable("grasp_1", 3, -1., 1.)
#bdr1.add_variable("qtn_ho", 4, -1., 1.)
#bdr1.add_variable("pos_ho", 3, ws_lb[:3], ws_ub[:3])
obj_pose_init = obj_start.pose.parameters()
bdr1.add_variable("posevec_ho", 6, ho_lb - to_posevec(obj_pose_init) , ho_ub - to_posevec(obj_pose_init))
bdr1.add_parameter("pose_o0", 7)
bdr1.add_parameter("pose_og", 7)
bdr1.add_parameter("pose_r1", 7)
bdr1.add_parameter("pose_r2", 7)

bdr1.register_fn("grasp_fn", [3], 1,
    grasp_constr, jac_grasp_constr)
bdr1.register_fn("manip_fn", [3, 6, 7, 7, 7], 2,
    manip_constr_ec, jac_manip_constr, jac_out_argnums=[0, 1])
# bdr1.register_fn("qtn_constr", [4], 1,
#     qtn_constr, jac_qtn_constr)

In [138]:
#bdr1.set_objective("grasp_obj", ["grasp_0", "grasp_1"])
bdr1.set_constr("grasp_prob_pick", "grasp_fn", ["grasp_0"], 
                   1., np.inf)
bdr1.set_constr("grasp_prob_place", "grasp_fn", ["grasp_1"], 
                   1., np.inf)
bdr1.set_constr("manip_pick_ho", "manip_fn", 
                ["grasp_0", "posevec_ho", "pose_o0", "pose_r1", "pose_o0"], 
                 0.2, np.inf) #, no_deriv_names=["grasp_0"]
bdr1.set_constr("manip_place_ho", "manip_fn", 
                ["grasp_1", "posevec_ho", "pose_og", "pose_r2", "pose_o0"], 
                 0.2, np.inf)
# bdr1.set_constr("qtn_manifold", "qtn_constr", ["qtn_ho"], 0., 0.)

In [139]:
def debug_callback(xdict):
    grasp0 = xdict["grasp_0"]
    grasp1 = xdict["grasp_1"]
    # qtn_ho = xdict["qtn_ho"]
    # pos_ho = xdict["pos_ho"]
    posevec_ho = xdict["posevec_ho"]
    pose_ref = xdict["pose_o0"]
    #pose_ho = merge_pose(pos_ho, qtn_ho)
    pose_ho = SE3(pose_ref) @ to_pose(posevec_ho)
    pose_o0 = SE3(xdict["pose_o0"])
    pose_og = SE3(xdict["pose_og"])
    #print(f"pos:{pose_ho.parameters()[-3:]}")    
    obj.set_pose(pose_ho)
    grasp_pose1_ho0 = pose_ho @ grasp_reconst(grasp0)
    grasp_pose1_o0 = pose_o0 @ grasp_reconst(grasp0)
    grasp_pose1_ho1 = pose_ho @ grasp_reconst(grasp1)
    grasp_pose1_o1 = pose_og @ grasp_reconst(grasp1)
    hand1.set_pose(grasp_pose1_ho0)
    hand2.set_pose(grasp_pose1_o0)
    hand3.set_pose(grasp_pose1_ho1)
    hand4.set_pose(grasp_pose1_o1)
    #print(f"err: {qtn_err(qtn, qtn_d)}  viol:{qtn_constr(qtn)}")
    time.sleep(0.5)
bdr1.set_debug_callback(debug_callback)

In [140]:
ipopt_grasp = bdr1.build()

compiling objective ...
compiling gradient ...
compiling constraints ...
compiling jacobian ...
ooo-------------------------------------
---ooo----------------------------------
ooo---oooooo----------------------------
ooo---oooooo----------------------------
---ooooooooo----------------------------
---ooooooooo----------------------------


In [141]:
bdr1.print_x_info()

{'grasp_0': 3, 'grasp_1': 3, 'posevec_ho': 6, 'pose_o0': 7, 'pose_og': 7, 'pose_r1': 7, 'pose_r2': 7}


In [142]:
grasp = jnp.zeros(3)
pose_o = obj_start.pose.parameters()
pose_g = obj_goal.pose.parameters()
#pose_r = SE3.identity().parameters()
pose_r1 = SE3.from_translation(jnp.array([0, -0.4, 0])).parameters()
pose_r2 = SE3.from_translation(jnp.array([0, 0.4, 0])).parameters()
pose_oref = pose_o
posevec_ho = to_posevec((SE3(pose_oref).inverse() @ obj_ho.pose).parameters())
x0 = np.hstack([grasp, grasp, posevec_ho, pose_o, pose_g, pose_r1, pose_r2])
grasp_sol, info = ipopt_grasp.solve(x0)


This is Ipopt version 3.14.10, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       42
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       40
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       12
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.81e+02 1.00e+00   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [127]:
hand3.set_pose(SE3.identity())

In [77]:
in_dims = [1, 3, 2, 4]
np.arange(len(in_dims))

array([0, 1, 2, 3])

Array(0.37309918, dtype=float32)